[Sascha Spors](https://orcid.org/0000-0001-7225-9992),
Professorship Signal Theory and Digital Signal Processing,
[Institute of Communications Engineering (INT)](https://www.int.uni-rostock.de/),
Faculty of Computer Science and Electrical Engineering (IEF),
[University of Rostock, Germany](https://www.uni-rostock.de/en/)

# Tutorial Signals and Systems (Signal- und Systemtheorie)

Summer Semester 2023 (Bachelor Course #24015)

- lecture: https://github.com/spatialaudio/signals-and-systems-lecture
- tutorial: https://github.com/spatialaudio/signals-and-systems-exercises

Feel free to contact lecturer [frank.schultz@uni-rostock.de](https://orcid.org/0000-0002-3010-0294)

## Übung / Exercise 2 Faltung / Convolution : Animation

rect / tri

source code adapted from

https://github.com/spatialaudio/signals-and-systems-lecture/commit/72652fc12b1527698db2e4543d4a16b211526aa2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sympy as sp

plt.rcParams['animation.html'] = 'jshtml'

In [ ]:
def heaviside(x):
    return np.heaviside(x, .5)


def animate_convolution(x, h, y, td, taud, xlim):

    # add numerical evaluation to symbolic functions
    x_eval = sp.lambdify((tau), x.subs(t, tau), modules=[
                         'numpy', {'Heaviside': heaviside}])
    h_eval = sp.lambdify((t, tau), h.subs(t, t - tau),
                         modules=['numpy', {'Heaviside': heaviside}])
    y_eval = sp.lambdify((t), y, modules=['numpy', {'Heaviside': heaviside}])

    # setup plot and line styles
    fig, ax = plt.subplots(2, 1)
    fig.subplots_adjust(hspace=0.5)
    plt.close()  # suppresses empty plot in notebook

    lines = [ax[0].plot([], [], lw=2, label=r'$x_B(t-\tau)$')[0]]
    lines.append(ax[0].plot([], [], lw=2, label=r'$x_A(\tau)$')[0])
    lines.append(ax[1].plot([], [], 'g-', lw=2,
                 label=r'$y(t) = x_A(t) * x_B(t)$')[0])
    lines.append(ax[1].plot([], [], 'ro', lw=2)[0])

    ax[0].set_xlim(xlim)
    ax[0].set_ylim((-0.1, 2.1))
    ax[0].set_xlabel(r'$\tau$')
    ax[0].legend(loc='upper right')
    ax[0].grid(True)

    ax[1].set_xlim(xlim)
    ax[1].set_ylim((-0.1, 1))
    ax[1].set_xlabel(r'$t$')
    ax[1].legend(loc='upper right')
    ax[1].grid(True)

    if False:  # check manual calc
        t1 = np.linspace(1, 3/2, 100)
        x1 = -2*t1**2 + 6*t1 - 4
        ax[1].plot(t1, x1, 'C3')

        t2 = np.linspace(3/2, 3, 100)
        x2 = t2*0+1/2
        ax[1].plot(t2, x2, 'C3')

        t3 = np.linspace(3, 7/2, 100)
        x3 = +2*t3**2 - 14*t3 + 49/2
        ax[1].plot(t3, x3, 'C3')

    def animate(ti):
        ax[0].collections.clear()
        ax[0].fill_between(taud, 0, h_eval(ti, taud) *
                           x_eval(taud), facecolor='gray', alpha=0.3)

        lines[0].set_data(taud, h_eval(ti, taud))
        lines[1].set_data(taud, x_eval(taud))

        lines[2].set_data(taud, y_eval(taud))
        lines[3].set_data(ti, y_eval(ti))

        return lines

    return animation.FuncAnimation(fig, animate, td, interval=50, blit=True)

In [ ]:
class rect(sp.Function):

    @classmethod
    def eval(cls, arg):
        return sp.Heaviside(arg + sp.S.Half) - sp.Heaviside(arg - sp.S.Half)


t, tau = sp.symbols('t tau', real=True)

## UE 2.6

For the given signals

\begin{equation}
h(t) =
\begin{cases}
-2 t + 3 \quad \mathrm{for} \quad 1 \leq t \leq \frac{3}{2}\\
0 \quad \mathrm{else}
\end{cases}
\end{equation}
\begin{equation}
h(t) = (-2 t + 3) \cdot \mathrm{rect}\left(\left[t-\frac{5}{4}\right] \cdot 2\right)
\end{equation}
and
\begin{equation}
x(t)=
\begin{cases}
  2 \quad \mathrm{for} \quad 0 \leq t \leq 2\\
  0 \quad \mathrm{else}
\end{cases}
\end{equation}
\begin{equation}
x(t) = 2\cdot\mathrm{rect}\left(\left[t-1\right] \cdot \frac{1}{2}\right)
\end{equation}
the convolution
\begin{equation}
y(t) = h(t) \ast x(t) = x(t) \ast h(t)
\end{equation}
is to be calculated analytically. The signals $h(t)$, $x(t)$ and $y(t)$ are to be visualized.


In [ ]:
x = 2 * rect((t-1)*1/2)
h = (-2*t + 3) * rect((t-5/4)*2)

In [ ]:
xA = x
xB = h
y = sp.integrate(xA.subs(t, t-tau) * xB.subs(t, tau),
                 (tau, 0, t))  # for causal signals

anim = animate_convolution(
    x=xA,
    h=xB,
    y=y,
    td=np.arange(-2, 10, .1),
    taud=np.arange(-10, 10, 0.01),
    xlim=(-2, 5))
anim

$$y(t) = \int x_A(\tau) \cdot x_B(-\tau + t) \mathrm{d}\tau$$

$$y(t) = \int x(\tau) \cdot h(-\tau + t) \mathrm{d}\tau$$

In [ ]:
xA = h
xB = x
y = sp.integrate(xA.subs(t, t-tau) * xB.subs(t, tau),
                 (tau, 0, t))  # for causal signals

anim = animate_convolution(
    x=xA,
    h=xB,
    y=y,
    td=np.arange(-2, 10, .1),
    taud=np.arange(-10, 10, 0.01),
    xlim=(-2, 5))
anim

$$y(t) = \int x_A(\tau) \cdot x_B(-\tau + t) \mathrm{d}\tau$$

$$y(t) = \int h(\tau) \cdot x(-\tau + t) \mathrm{d}\tau$$

## Copyright

This tutorial is provided as Open Educational Resource (OER), to be found at
https://github.com/spatialaudio/signals-and-systems-exercises
accompanying the OER lecture
https://github.com/spatialaudio/signals-and-systems-lecture.
Both are licensed under a) the Creative Commons Attribution 4.0 International
License for text and graphics and b) the MIT License for source code.
Please attribute material from the tutorial as *Frank Schultz,
Continuous- and Discrete-Time Signals and Systems - A Tutorial Featuring
Computational Examples, University of Rostock* with
``github URL, commit number and/or version tag, year, (file name and/or content)``.